## 사용한 패키지 버전
pandas                        1.4.2
numpy                         1.21.5

In [ ]:
import numpy as np
import pandas as pd

loan=pd.read_csv('loan_result.csv')
user=pd.read_csv('user_spec.csv')

## user_spec 전처리

In [ ]:
# 입사연월을 년+월로 바꾸기
# 입사연월을 string으로 바꾸고
user['company_enter_month'] = user['company_enter_month'].astype(str) 
print(user['company_enter_month'].dtypes)

# 앞에서 6자리만 끊기 (년+월) 만
user['company_enter_month'] = user['company_enter_month'].str[:6]

# 다시 object를 숫자(int)형으로
user['company_enter_month'] = user['company_enter_month'].apply(pd.to_numeric, errors = 'coerce')

# purpose 한글로 통합
user=user.replace({'purpose':'LIVING'},'생활비')
user=user.replace({'purpose':'BUSINESS'},'사업자금')
user=user.replace({'purpose':'BUYCAR'},'자동차구입')
user=user.replace({'purpose':'BUYHOUSE'},'주택구입')
user=user.replace({'purpose':'ETC'},'기타')
user=user.replace({'purpose':'HOUSEDEPOSIT'},'전월세보증금')
user=user.replace({'purpose':'INVEST'},'투자')
user=user.replace({'purpose':'SWITCHLOAN'},'대환대출')

In [ ]:
user.loc[(user['personal_rehabilitation_yn']== 0) & (user['personal_rehabilitation_complete_yn']=='NaN'), 'personal_rehabilitation_complete_yn'] = 0

In [ ]:
re=user[(user['personal_rehabilitation_yn']==0) & (user['personal_rehabilitation_complete_yn'].isnull())].index
user['personal_rehabilitation_complete_yn'].loc[re]=0

user['insert_time']=pd.to_datetime(user['insert_time'])

## loan_result 전처리

In [ ]:
# '한도조회 일시' 날짜 형식으로 변환
loan['loanapply_insert_time']=pd.to_datetime(loan['loanapply_insert_time'])

In [ ]:
# 데이터의 중복성 확인
loan.duplicated(keep=False).any()

In [ ]:
# 중복 행 df 크기 확인
loan[loan.duplicated(keep = False)].shape

In [ ]:
# train data에 해당하는 중복행
loan[(loan['is_applied'].notnull()) & (loan.duplicated(keep=False))]

In [ ]:
# train data에 해당하는 중복행 제거
du=loan[(loan.duplicated(keep=False)) & (loan['is_applied'].notnull())].drop_duplicates(keep='first').index
loan=loan.drop(du, axis=0)


## 데이터 병합

In [ ]:
loan_user=pd.merge(loan, user, how='left', on='application_id')
# loan_limit, loan_rate결측치 삭제
loan_user.dropna(subset=['loan_limit','loan_rate'], inplace=True)

In [ ]:
loan_user.to_csv('loan_user_merge_first.csv')